# 🏇 競馬AI データパイプライン（Google Colab版）

このノートブックは以下を実行します：
1. **スクレイピング**: netkeiba.comからレースデータを収集
2. **データ前処理**: 欠損値処理、型変換、外れ値除去
3. **特徴量生成**: 過去走データから予測用特徴量を計算
4. **SQLite保存**: データベースに保存してアプリから利用可能に

## 📋 必要なライブラリ

In [ ]:
# Google Colab環境のセットアップ
!pip install beautifulsoup4 requests pandas numpy scikit-learn lightgbm tqdm -q

import os
import sys
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from tqdm import tqdm
import pickle
import json

## 🔧 Google Driveマウント（オプション）

データベースをGoogle Driveに保存する場合

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 作業ディレクトリの設定
WORK_DIR = '/content/drive/MyDrive/keiba-ai'
os.makedirs(WORK_DIR, exist_ok=True)
os.chdir(WORK_DIR)

print(f"✅ 作業ディレクトリ: {WORK_DIR}")

## 📥 GitHubからコードをクローン

In [ ]:
# GitHubリポジトリのクローン（初回のみ）
REPO_URL = "https://github.com/YOUR_USERNAME/dai-keiba.git"  # 実際のURLに変更

if not os.path.exists('dai-keiba'):
    !git clone {REPO_URL}
    print("✅ リポジトリをクローンしました")
else:
    print("✅ リポジトリは既に存在します")
    # 最新版を取得
    !cd dai-keiba && git pull

# パスの追加
sys.path.append('dai-keiba')
sys.path.append('dai-keiba/scraper')
sys.path.append('dai-keiba/ml')

## 🕷️ ステップ1: スクレイピング

### JRAデータの収集

In [ ]:
from scraper import auto_scraper

# 設定
START_DATE = "2023-01-01"  # 開始日
END_DATE = datetime.now().strftime("%Y-%m-%d")  # 今日まで
MODE = "JRA"  # JRA または NAR

print(f"📅 収集期間: {START_DATE} ～ {END_DATE}")
print(f"🏇 モード: {MODE}")
print("\n⏳ スクレイピング開始...（数時間かかる場合があります）")

# スクレイピング実行
# 注: auto_scraperのscrape_historical_data関数を呼び出す
# 詳細は dai-keiba/ml/scrape_historical_data.py を参照

# 暫定的にCSVとして保存
csv_path = f"database_{MODE.lower()}.csv"

# ここに実際のスクレイピング処理を実装
# 例: df = auto_scraper.scrape_range(START_DATE, END_DATE, MODE)
# df.to_csv(csv_path, index=False)

print(f"✅ スクレイピング完了: {csv_path}")

## 🔄 ステップ2: 特徴量生成

In [ ]:
from ml.feature_engineering import calculate_features

# 設定
USE_VENUE_FEATURES = True  # 会場特性特徴量を使用

input_csv = f"database_{MODE.lower()}.csv"
output_csv = f"processed_data_{MODE.lower()}.csv"

print(f"⚙️ 特徴量生成開始...")
print(f"   入力: {input_csv}")
print(f"   出力: {output_csv}")
print(f"   会場特性: {USE_VENUE_FEATURES}")

# 特徴量生成
calculate_features(
    input_csv,
    output_csv,
    lambda_decay=0.2,
    use_venue_features=USE_VENUE_FEATURES
)

print(f"✅ 特徴量生成完了: {output_csv}")

## 💾 ステップ3: SQLiteデータベースに保存

In [ ]:
# SQLiteデータベースの作成
DB_PATH = "keiba_data.db"

print(f"💾 データベース作成: {DB_PATH}")

# データの読み込み
df_processed = pd.read_csv(output_csv)

print(f"   レコード数: {len(df_processed)}")
print(f"   特徴量数: {len(df_processed.columns)}")

# SQLite接続
conn = sqlite3.connect(DB_PATH)

# テーブル名の決定
table_name = f"processed_data_{MODE.lower()}"

# DataFrameをSQLiteに保存
df_processed.to_sql(
    table_name,
    conn,
    if_exists='replace',  # 既存テーブルを置き換え
    index=False
)

# インデックスの作成（高速化）
cursor = conn.cursor()
cursor.execute(f"CREATE INDEX IF NOT EXISTS idx_race_id ON {table_name}(race_id)")
cursor.execute(f"CREATE INDEX IF NOT EXISTS idx_horse_id ON {table_name}(horse_id)")
cursor.execute(f"CREATE INDEX IF NOT EXISTS idx_date ON {table_name}(date)")
conn.commit()

print(f"✅ データベース保存完了: {table_name}")

# 確認クエリ
result = pd.read_sql_query(f"SELECT COUNT(*) as count FROM {table_name}", conn)
print(f"   保存されたレコード数: {result['count'].iloc[0]}")

conn.close()

## 📊 ステップ4: データ検証

In [ ]:
# データベースからサンプルを読み込んで確認
conn = sqlite3.connect(DB_PATH)

# テーブル一覧
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn)
print("📋 テーブル一覧:")
print(tables)

# サンプルデータ
print("\n📊 サンプルデータ（先頭5件）:")
sample = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT 5", conn)
print(sample)

# 統計情報
print("\n📈 統計情報:")
stats = pd.read_sql_query(f"""
    SELECT 
        COUNT(*) as total_records,
        COUNT(DISTINCT race_id) as unique_races,
        COUNT(DISTINCT horse_id) as unique_horses,
        MIN(date) as earliest_date,
        MAX(date) as latest_date,
        AVG(target_win) as win_rate
    FROM {table_name}
""", conn)
print(stats)

conn.close()

## 📤 ステップ5: データベースのダウンロード

作成したデータベースをローカルにダウンロードして、アプリケーションで使用します。

In [ ]:
from google.colab import files

# データベースファイルをダウンロード
print("📥 データベースをダウンロード中...")
files.download(DB_PATH)
print(f"✅ ダウンロード完了: {DB_PATH}")
print("\n👉 このファイルをプロジェクトのルートディレクトリに配置してください")

## 🔄 NAR（地方競馬）の処理

NARデータも同様に処理する場合は、MODEを変更して再実行してください。

In [ ]:
# NAR処理の例
MODE_NAR = "NAR"

# 上記のセルをMODE="NAR"で再実行するか、
# または関数化して呼び出す

def process_race_data(mode="JRA", use_venue_features=True):
    """レースデータの処理パイプライン"""
    print(f"\n{'='*60}")
    print(f"モード: {mode}")
    print(f"{'='*60}")
    
    # 1. スクレイピング（省略）
    
    # 2. 特徴量生成
    input_csv = f"database_{mode.lower()}.csv"
    output_csv = f"processed_data_{mode.lower()}.csv"
    
    calculate_features(
        input_csv,
        output_csv,
        lambda_decay=0.2,
        use_venue_features=use_venue_features
    )
    
    # 3. SQL保存
    df = pd.read_csv(output_csv)
    conn = sqlite3.connect(DB_PATH)
    table_name = f"processed_data_{mode.lower()}"
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.close()
    
    print(f"✅ {mode}データの処理完了")

# JRAとNARの両方を処理
# process_race_data("JRA", use_venue_features=True)
# process_race_data("NAR", use_venue_features=True)

## ✅ 完了

これで以下が完了しました：
1. ✅ スクレイピング
2. ✅ 特徴量生成
3. ✅ SQLiteデータベース作成
4. ✅ データ検証

次のステップ：
1. `keiba_data.db`をダウンロード
2. プロジェクトのルートディレクトリに配置
3. 管理画面でモデルを学習
4. 公開ページで予測を実行